In [1]:
import numpy as np
# import sys
import os
import scipy.io
# option to import from github folder
# sys.path.insert(0, 'E:/Users/samsoon.inayat/T_Drive/GitHub/Downloaded/suite2p/')
# from suite2p import run_s2p
import suite2p
import Thor_Experiment
from suite2p import run_s2p, default_ops
import shutil

In [2]:
# set your options for running
ops = default_ops() # populates ops with the default options
# print(ops)

In [3]:
# tif data folder where tif files will be extracted from raw data
tif_data_folder = 'E:/Users/samsoon.inayat/S_Drive/Tif_Data'
# processed_data_folder where the results of suite 2p will be stored
#processed_data_folder = 'E:/Users/samsoon.inayat/OneDrive - University of Lethbridge/pySuite2P/processed_data'
processed_data_folder = 'E:/Users/samsoon.inayat/S_Drive/Processed_Data'
nas_processed_data_folder = 'Z:/homes/brendan.mcallister/2P/Processed_Data_1'

In [4]:
filename = 'recording_list.txt'
f = open(filename,'r')
dir_names = []
for drs in f:
    temp = drs[:-1]
    tempp = temp.replace(os.sep,os.altsep)
    dir_names.append(tempp)
f.close()

In [5]:
print(dir_names)

['//mohajerani-nas.uleth.ca/storage/homes/samsoon.inayat/Data/183633/2019-06-04/1_001', '//mohajerani-nas.uleth.ca/storage/homes/samsoon.inayat/Data/183761/2019-06-06/1_002', '//mohajerani-nas.uleth.ca/storage/homes/samsoon.inayat/Data/183745/2019-06-07/1_001', '//mohajerani-nas.uleth.ca/storage/homes/samsoon.inayat/Data/183628/2019-06-11/1_001', '//mohajerani-nas.uleth.ca/storage/homes/samsoon.inayat/Data/183762/2019-06-11/1_001']


In [ ]:
for x in dir_names:
    print(x)
    dir_name = x
    te = Thor_Experiment.Thor_Exp(dir_name,processed_data_folder,tif_data_folder,nas_processed_data_folder)
    data_processing_status = 0
    for root, dirs, files in os.walk(te.nas_pd_dir):
        for name in files:
            if name.endswith(("Fall.mat")):
                data_processing_status = 1
                break
    if data_processing_status == 1:
        continue
#    if te.exp_params.get('zFastEnable') == '1':
#        continue
#     ops = default_ops()
#        print(ops)
    ops.update({'nplanes':te.exp_params.get('nplanes')})
    ops.update({'save_path0':te.pd_dir})
    ops.update({'fs':float(te.exp_params.get('frameRate'))})
    ops.update({'save_mat':True})
    mat = scipy.io.loadmat(te.nas_pd_dir + '/bidishift.mat')
    bidishift = mat['bidishift'];
#    print(bidishift[0][0])
#    break
    ops.update({'do_bidiphase':True})
    ops.update({'bidiphase':bidishift[0][0]})
    ops.update({'roidetect':True})
    ops.update({'do_registration':1})
    ops.update({'tau':1.5})
    print(ops)
    db = {
            'h5py': [], # a single h5 file path
            'h5py_key': 'data',
            'data_path': [te.tif_dir_name], # a list of folders with tiffs
                                         # (or folder of folders with tiffs if look_one_level_down is True, or subfolders is not empty)
            }
    np.save('ops.npy', ops)
    np.save('db.npy', db)
#     subprocess.call(['python','-u','-W','ignore','-m','suite2p','--ops', 'ops.npy','--db', 'db.npy'],shell=True)
#    break
    opsEnd = run_s2p(ops = ops,db = db)
    if os.path.exists(te.tif_dir_name):
        shutil.rmtree(te.tif_dir_name)
    else:
        print('tif folder already removed')
    del(te)